<h1  align="center"><b> MODELO AutoARIMA </b></h1>

`Objetivo Geral:` Importar a série temporal transformada dos dados pluviométricos do município de São Paulo e realizar a modelagem AutoARIMA para previsão.

`Dados:` Os dados foram transformados na pasta [Transformação e Decomposição](../[3]%20Transformação%20e%20Decomposição%20-%20Projeto%20Chuva/) do Projeto Chuva.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6

<h2 align="center"><b> Importando Dados e Escolhendo o Modelo </b></h2>
&emsp;&emsp; O modelo AutoARIMA é um modelo bem legal, pois ele não só escolhe os melhores parâmetros para o modelo ARIMA, como também considera a sazonalidade da série temporal (SARIMA).

`Observação:` SARIMA(p,d,q)(P, D, Q)

In [3]:
série_chuva = pd.read_csv('../[3] Transformação e Decomposição - Projeto Chuva/Série Transformada - Chuva Mensal.csv', sep = ';', index_col = 0)
série_chuva = pd.Series(série_chuva['Chuva Mensal (mm)'])
série_chuva.index = pd.date_range('1985', periods = len(série_chuva), freq = 'M')

`trace:` Se True, printa o modelo de cada iteração.

`stepwise:` Se True, usa o método de busca stepwise para escolher o melhor modelo. Esse método é mais rápido, porém não garante o melhor modelo.

`seasonal:` Se True, considera a sazonalidade da série temporal (SARIMA).

`m:` Número de períodos da sazonalidade.

In [7]:
from pmdarima import auto_arima
modelo_AutoARIMA = auto_arima(série_chuva, trace = True, stepwise = True, seasonal = True, stationary = True, m = 12)

Performing stepwise search to minimize aic
 ARIMA(2,0,2)(1,0,1)[12] intercept   : AIC=inf, Time=3.55 sec
 ARIMA(0,0,0)(0,0,0)[12] intercept   : AIC=1706.610, Time=0.09 sec
 ARIMA(1,0,0)(1,0,0)[12] intercept   : AIC=1491.065, Time=1.42 sec
 ARIMA(0,0,1)(0,0,1)[12] intercept   : AIC=1545.043, Time=0.66 sec
 ARIMA(0,0,0)(0,0,0)[12]             : AIC=2768.434, Time=0.06 sec
 ARIMA(1,0,0)(0,0,0)[12] intercept   : AIC=1577.903, Time=0.18 sec
 ARIMA(1,0,0)(2,0,0)[12] intercept   : AIC=1452.351, Time=4.51 sec
 ARIMA(1,0,0)(2,0,1)[12] intercept   : AIC=inf, Time=8.56 sec
 ARIMA(1,0,0)(1,0,1)[12] intercept   : AIC=inf, Time=1.37 sec
 ARIMA(0,0,0)(2,0,0)[12] intercept   : AIC=1454.125, Time=2.71 sec
 ARIMA(2,0,0)(2,0,0)[12] intercept   : AIC=1451.895, Time=7.35 sec
 ARIMA(2,0,0)(1,0,0)[12] intercept   : AIC=1488.499, Time=3.25 sec
 ARIMA(2,0,0)(2,0,1)[12] intercept   : AIC=inf, Time=10.24 sec
 ARIMA(2,0,0)(1,0,1)[12] intercept   : AIC=inf, Time=3.00 sec
 ARIMA(3,0,0)(2,0,0)[12] intercept   : AIC=

In [8]:
resultado_AutoARIMA = modelo_AutoARIMA.fit(série_chuva) # Resultado: ARIMA (2, 0, 4)
print(resultado_AutoARIMA.summary())

                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                  456
Model:               SARIMAX(2, 0, 4)   Log Likelihood                -658.835
Date:                Sun, 24 Dec 2023   AIC                           1333.670
Time:                        19:43:03   BIC                           1366.650
Sample:                    01-31-1985   HQIC                          1346.661
                         - 12-31-2022                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      1.2770      0.015     83.675      0.000       1.247       1.307
ar.L1          1.7297      0.002    814.085      0.000       1.726       1.734
ar.L2         -0.9978      0.002   -499.922      0.0

<h2 align="center"><b> Análise de Resíduos </b></h2>

In [ ]:
resíduo = resultado_AutoARIMA.resid() # Obtendo os resíduos
resíduo.plot() # Plotando o resíduo
plt.show()

![AutoARIMA_Residuos](./Gráficos/AutoARIMA_Residuos.png)

### `Normalidade dos Resíduos:`

In [10]:
import scipy.stats as stats
import seaborn as sns

In [ ]:
stats.probplot(resíduo, dist = 'norm', plot = plt)
plt.title('Normal QQ Plot - Resíduos')
plt.show()

sns.histplot(resíduo, kde = True)
plt.title('Histograma - Resíduos')
plt.show()

![Normal QQ Plot - Resíduos](./Gráficos/Normal%20QQ%20Plot%20-%20Resíduos.png)
![Histograma - Resíduos](./Gráficos/Histograma%20-%20Resíduos.png)

In [12]:
def teste_shapiro(série):
    e, p = stats.shapiro(série)
    print(f'Estatística de Teste = {e}')
    print(f'p-valor = {p}')
    print(f'Resultado: {"É Normal" if (p > 0.05) else "Não é Normal"}')

teste_shapiro(resíduo)

Estatística de Teste = 0.989251434803009
p-valor = 0.0019987672567367554
Resultado: Não é Normal


&emsp;&emsp; Esse modelo ficou melhor que o melhor modelo que tínhamos até então (ARMA(2, 2), p-valor = 0.01008275). A distribuição ainda não é normal, mas se aproxima bastante. 

### `Autocorrelação dos Resíduos:`

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

plot_acf(resíduo, lags = 20)
plt.title('Função de Autocorrelação - Resíduos')
plt.show()

plot_pacf(resíduo, lags = 20)
plt.title('Função de Autocorrelação Parcial - Resíduos')
plt.show()

![Função de Autocorrelação - Resíduos](./Gráficos/Função%20de%20Autocorrelação%20-%20Resíduos.png)
![Função de Autocorrelação Parcial - Resíduos](./Gráficos/Função%20de%20Autocorrelação%20Parcial%20-%20Resíduos.png)

&emsp;&emsp; As funções de autocorrelação mostram que os resíduos não possuem correlação com os valores passados, o que é um ótimo sinal.

<h2 align="center"><b> Previsão </b></h2>

In [14]:
tamanho_série = len(série_chuva) # Tamanho da série
previsão = resultado_AutoARIMA.predict(n_periods = 12) # Previsão para os próximos 12 meses

In [ ]:
plt.plot(série_chuva, label = 'Série Original')
plt.plot(série_chuva - resíduo, label = 'Resíduo')
plt.plot(previsão, label = 'Previsão')
plt.title('Previsão - Modelo AutoARIMA')
plt.legend(loc = 'best')
plt.show()

![Previsão - Modelo AutoARIMA](./Gráficos/Previsão%20-%20Modelo%20AutoARIMA.png)

&emsp;&emsp; Finalmente chegamos na previsão, mas esses valores passaram por uma transformação por raiz cúbica, então precisamos elevar os valores ao cubo para obter a previsão real.

<h2 align="center"><b> Finalização </b></h2>

In [19]:
previsão_final = previsão ** 3
previsão_final.name = 'Previsão AutoARIMA'
previsão_final.to_csv('Previsão AutoARIMA - Chuva Mensal.csv', sep = ';', header = True) # Salvando a previsão em um arquivo csv
display(previsão_final)

2023-01-31    250.034300
2023-02-28    212.695357
2023-03-31    150.615171
2023-04-30     92.760150
2023-05-31     54.796134
2023-06-30     36.931739
2023-07-31     34.246728
2023-08-31     45.550820
2023-09-30     74.841341
2023-10-31    125.218512
2023-11-30    187.714971
2023-12-31    237.072483
Freq: M, Name: Previsão AutoARIMA, dtype: float64